<a href="https://colab.research.google.com/github/tritus-soi/sample_project/blob/master/Service_provider.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install geopandas

In [ ]:
pip install folium

In [ ]:
import pandas as pd
import geopandas as gpd
import folium

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## จำนวนอู่ รถยก รถสไสด์ ช่างที่เคยติดต่อ

In [ ]:
listing_df = pd.read_csv('/content/drive/MyDrive/Wsmart/sp_db.csv')
listing_df.head(5)

,en_city_name,region,contract,price_rate,registered_name,line_id_name,district,province,service_area,tel_1,tel_2,service_type,emergency,night_service,tow_0-15,tow_16-30,tow_31-50,tow_excess,slide_0-15,slide_16-30,slide_31-50,slide_excess,jump_start_0-15,jump_start_16-30,jump_start_excess,tire_change_0-15,tire_change_16-30,tire_change_excess,fuel_refill_0-15,fuel_refill_16-30,fuel_refill_excess,onsite_0-15,onsite_16-30,onsite_excess,unlock_0-15,unlock_16-30,unlock_excess,long_trip,remove_code,oil_change,basic_check,rating,comment,remark
0,Bangkok Metropolis,BMR,True,B,ฐานุตร์ ธนบุญสุทธิ BT Batterry Delivery กทม,BT Batterry Delivery,NaN,กรุงเทพมหานคร,NaN,088-005-1629,NaN,ร้านแบตเตอรี่,RSA,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400,400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bangkok Metropolis,BMR,True,B,บังมัท รถยก - รถสไลด์,รถยก รถสไลด์บังมัท&พี่ชาติ,ประเวศ,กรุงเทพมหานคร,"อุดมสุข, ประเวศ, ลาดกระบัง, หลวงแพ่ง, สนามบินส...",088-891-6225,พี่ชาติ 061-480-7449,รถยก/รถสไลด์,RSA,YES,1000,1500,NaN,25,1500,1900,NaN,25,NaN,NaN,NaN,NaN,NaN,NaN,300,500,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bangkok Metropolis,BMR,True,B,นาย สุรพล เคยสนิท,สุรพล รถยก,ดอนเมือง,กรุงเทพมหานคร,"ดอนเมือง, วิภาวดี, ปทุมธานี, คลองหลวง, ปากเกร็...",088-492-0534,088-4920534,รถยก/รถสไลด์,RSA,YES,900,1200,1300,20,1300,1500,1700,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bangkok Metropolis,BMR,True,C,จิระเดช ชิตะวณิช ( ช่าง เฟิรส 0851458488 จ.กรุ...,ช่าง First,บางพลัด,กรุงเทพมหานคร,"บางกรวย, บางพลัด, ปิ่นเกล้า, รัชดาภิเษก, ลาดพร...",085-145-8488,085-1458488,ร้านแบตเตอรี่,RSA,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400,600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Bangkok Metropolis,BMR,True,A,ช่างเนตรป๋อ,เนรตป๋อ,ประเวศ,กรุงเทพมหานคร,"ประเวศ, หนองบอน",080-055-7150,080-055-7150,อู่ซ่อมทั่วไป/ตรวจเช็ค/เติมน้ำมัน,RSA - Non RSA,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
nbh_geo_df = gpd.read_file('https://gist.githubusercontent.com/jeepkd/4e31e6a10f8297b9de50c62856927ecf/raw/9899d9f1ca4cd7c5f103a9b2455d9a01f0c8f895/thailand.json', driver='GeoJSON')
nbh_geo_df = nbh_geo_df[['NAME_1', 'geometry',]]
nbh_geo_df

,ID_0,ISO,NAME_0,ID_1,NAME_1,TYPE_1,ENGTYPE_1,NL_NAME_1,VARNAME_1,geometry
0,228,THA,Thailand,1,Amnat Charoen,Changwat,Province,จังหวัดอำนาจเจริ,None,"POLYGON ((104.98066 16.27747, 105.01581 16.244..."
1,228,THA,Thailand,2,Ang Thong,Changwat,Province,จังหวัดอ่างทอง,None,"POLYGON ((100.46979 14.71473, 100.49653 14.697..."
2,228,THA,Thailand,3,Bangkok Metropolis,Changwat,Province,จังหวัดเชียงใหม่,Bangkok|Krung Thep|Krung Thep Maha Nakhon|Phra...,"POLYGON ((100.91515 13.94426, 100.90740 13.859..."
3,228,THA,Thailand,4,Bueng Kan,Changwat,Province,บึงกาฬ,None,"POLYGON ((104.19147 18.02050, 104.14795 18.012..."
4,228,THA,Thailand,5,Buri Ram,Changwat,Province,จังหวัดบุรีรัมย์,Buri Rum,"POLYGON ((103.11443 15.47934, 103.10193 15.427..."


In [ ]:
nbh_count_df = listing_df.groupby('en_city_name')['registered_name'].nunique().reset_index()
nbh_count_df.rename(columns={'registered_name':'nb'}, inplace=True)
nbh_geo_count_df = pd.merge(nbh_geo_df, nbh_count_df, left_on='NAME_1', right_on="en_city_name", how='left')
nbh_geo_count_df['QP'] = nbh_geo_count_df['nb'] / nbh_geo_count_df['nb'].sum()
nbh_geo_count_df['QP_str'] = nbh_geo_count_df['QP'].apply(lambda x : str(round(x*100, 1)) + '%')
nbh_geo_count_df.head(5)

,NAME_1,geometry,en_city_name,nb,QP,QP_str
0,Amnat Charoen,"POLYGON ((104.98066 16.27747, 105.01581 16.244...",Amnat Charoen,9,0.002738,0.3%
1,Ang Thong,"POLYGON ((100.46979 14.71473, 100.49653 14.697...",Ang Thong,10,0.003042,0.3%
2,Bangkok Metropolis,"POLYGON ((100.91515 13.94426, 100.90740 13.859...",Bangkok Metropolis,629,0.191360,19.1%
3,Bueng Kan,"POLYGON ((104.19147 18.02050, 104.14795 18.012...",Bueng Kan,12,0.003651,0.4%
4,Buri Ram,"POLYGON ((103.11443 15.47934, 103.10193 15.427...",Buri Ram,39,0.011865,1.2%


## จำนวนอู่ รถยกรถสไลด์ แยกรายจังหวัด เทียบเป็นร้อยละของทั้งหมด

In [ ]:
import folium

from branca.colormap import linear
nbh_count_colormap = linear.YlGnBu_09.scale(min(nbh_count_df['nb']),max(nbh_count_df['nb']))


m = folium.Map([15.4, 101.3], zoom_start=6, min_zoom=4, max_zoom=9, tiles='cartodbpositron')

style_function = lambda x: {
    'fillColor': nbh_count_colormap(x['properties']['nb']),
    'color': 'black',
    'weight': 1.5,
    'fillOpacity': 0.7
}

folium.GeoJson(
    nbh_geo_count_df,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['NAME_1', 'nb', 'QP_str'],
        aliases=['Province', 'Amount', 'Quote-part'],
        localize=True
    )
).add_to(m)

nbh_count_colormap.add_to(m)
nbh_count_colormap.caption = 'Service Provider'
nbh_count_colormap.add_to(m)

m